### Who Tweeted Most?

1. Group tweets by user
2. Count each user's tweets
3. Sort into descentding order
4. Select user at top

In [6]:
import json 
import pprint

def insert_data(data, db):
    for a in data:
        db.twitter.insert(a)

if __name__ == "__main__":
    
    from pymongo import MongoClient
    client = MongoClient("mongodb://localhost:27017")
    db = client.examples
    
    # Available here: http://content.udacity-data.com/ud032/twitter/twitter.json.zip
    with open('twitter.json', 'r') as f:
        ## json.loads() takes a string, while json.load() takes a file-like object.
        ## http://stackoverflow.com/questions/11568246/loading-several-text-files-into-mongodb-using-pymongo
        for tweet in f.readlines():
            db.twitter.insert(json.loads(tweet))
    pprint.pprint(db.twitter.find_one())

/Users/shuixin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


{'_id': ObjectId('5b101612a24d683bec191b47'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Sep 02 18:11:23 +0000 2010',
 'entities': {'hashtags': [], 'urls': [], 'user_mentions': []},
 'favorited': False,
 'geo': None,
 'id': 22819396900,
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_user_id': None,
 'place': None,
 'retweet_count': None,
 'retweeted': False,
 'source': 'web',
 'text': 'eu preciso de terminar de fazer a minha tabela, está muito foda **',
 'truncated': False,
 'user': {'contributors_enabled': False,
          'created_at': 'Fri Jul 03 21:44:05 +0000 2009',
          'description': 'só os loucos sabem (:',
          'favourites_count': 1,
          'follow_request_sent': None,
          'followers_count': 102,
          'following': None,
          'friends_count': 73,
          'geo_enabled': False,
          'id': 53507833,
          'lang': 'en',
          'listed_count': 0,
          'location': '',
          'nam

## $$$group,  $sort  $$

In [1]:
from pymongo import MongoClient
import pprint

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient("mongodb://localhost:27017")
    db = client[db_name]
    return db

In [2]:
def make_pipeline():
    pipeline = [{"$group": {"_id": "$source",
                            "count": {"$sum": 1}}},
                 {"$sort": {"count": -1}},
                 {"$limit" : 5 }]
    return pipeline

In [3]:
def tweet_sources(db, pipeline):
    result = db.tweets.aggregate(pipeline)
    return result

In [4]:
if __name__ == '__main__':
    db = get_db('example_tweets')
    pipeline = make_pipeline()
    result = tweet_sources(db, pipeline)
    import pprint
    pprint.pprint(result)

## $$$project,  $match,  $skip,  $limit,  $unwind  $$

### Who has the highest follower/friends ratio?

In [7]:
def highest_ratio():
    result = db.tweets.aggregate([
        {"$match": {"user.friends_count": {"$gt": 0},
                   "user.follower_count": {"$gt": 0}}},
        {"$project": {"ratio":{"$divide": ["$user.followers_count",
                                          "$user.friends_count"]},
                     "screen_name": "$user.screen_name"}},
        {"$sort": {"ratio": -1}},
        {"$limit": 1}
    ])
    
    return result

### Use $project to:
- Include fields from the original document
- Insert computed fields
- Rename fields
- Create fields that hold sub-documents